In [1]:
import spacy
from pytorch_pretrained_bert import BertTokenizer, BertForMaskedLM
from torch import Tensor
import torch
import re
import pandas as pd
pd.set_option('display.max_colwidth', -1)

In [19]:
tokenizer = BertTokenizer.from_pretrained('bert-large-uncased')
nouns = []
nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])

for token in tokenizer.vocab.keys():
    doc = nlp(token)
    if doc[0].pos_ == "NOUN":
        nouns.append(token)

In [20]:
bert = BertForMaskedLM.from_pretrained('bert-large-uncased')

In [21]:
emb = bert.bert.embeddings.word_embeddings.weight

In [22]:
emb = emb/emb.norm(dim = 1).reshape(-1,1)

In [23]:
_, similar_tokens_matrix = torch.mm(emb, emb.transpose(0,1)).topk(100)

In [24]:
df = pd.read_csv('../FilesToUploadToColab/test.txt', sep = '\t', header = None)
df.columns = ('rel', 'head','tail','label')

In [25]:
df.rel.value_counts()

AtLocation          479
IsA                 396
UsedFor             339
CapableOf           280
HasPrerequisite     226
HasSubevent         170
HasProperty         143
HasA                98 
Causes              93 
PartOf              52 
MadeOf              28 
ReceivesAction      22 
NotCapableOf        20 
Desires             17 
MotivatedByGoal     15 
CausesDesire        6  
CreatedBy           5  
NotHasProperty      4  
RelatedTo           3  
DefinedAs           2  
NotIsA              1  
HasFirstSubevent    1  
Name: rel, dtype: int64

In [36]:
def good_word(orig_w, new_w):
    if "#" in new_w:
        return False
    if orig_w[:-1] in new_w:
        return False
    if not new_w.isalpha():
        return False
    return True
relations = [
    'AtLocation',
    'IsA',            
    'HasPrerequisite',     
    'HasA',
    'PartOf'
]

In [37]:
count = 0
with open('./candidate_BERT.txt', 'w') as f:
    for n in nouns[115:165]:
        token = tokenizer.convert_tokens_to_ids([n])[0]
        similar_tokens = similar_tokens_matrix[token]
        similar_words = tokenizer.convert_ids_to_tokens(similar_tokens.tolist())[:10]
        good_words = [w for w in similar_words if good_word(n, w)]
        print(n)
        print(good_words)
        for idx, gw in enumerate(good_words):
            if idx > 5:
                continue
            for r in relations:
                count += 2
                f.write('\t'.join((r, n, gw, '2'))+'\n')
                f.write('\t'.join((r, gw, n, '2'))+'\n')

time
['day', 'it', 'one', 'the', 'period']
school
['college', 'university', 'house', 'church', 'work', 'education', 'academy', 'town']
years
['months', 'decades', 'days', 'weeks', 'seasons', 'centuries', 'decade', 'hours']
world
['international', 'life', 'city', 'global', 'country', 'universe', 'european', 'state']
year
['month', 'season', 'week', 'day', 'decade', 'summer', 'months', 'time']
city
['town', 'village', 'state', 'country', 'county', 'house', 'world', 'district']
state
['city', 'government', 'national', 'county', 'country', 'world', 'public', 'the']
people
['men', 'women', 'person', 'man', 'children', 'family', 'they']
part
['portion', 'one', 'half', 'section', 'a', 'all', 'side']
university
['college', 'school', 'institute', 'city', 'academy', 'campus', 'student', 'academic']
don
['didn', 'wouldn', 'can', 'con', 'won', 'dan', 'haven']
way
['far', 'time', 'path', 'means', 'place', 'road', 'manner']
head
['face', 'body', 'back', 'house', 'hand', 'chief']
team
['club', 'group

In [38]:
df_triples = pd.read_csv('./candidate_BERT.txt', sep = '\t', header= None, )
len(df_triples)

2940

In [39]:
df_triples

,0,1,2,3
0,AtLocation,time,day,2
1,AtLocation,day,time,2
2,IsA,time,day,2
3,IsA,day,time,2
4,HasPrerequisite,time,day,2
5,HasPrerequisite,day,time,2
6,HasA,time,day,2
7,HasA,day,time,2
8,PartOf,time,day,2
9,PartOf,day,time,2


In [40]:
df_triples.columns = ('rel','head','tail','label')

In [32]:
df = pd.read_csv('./BERT_mining_pred_direct.csv', index_col = 0)

FileNotFoundError: [Errno 2] File b'./BERT_mining_pred_direct.csv' does not exist: b'./BERT_mining_pred_direct.csv'

In [225]:
df_triples['prob'] =  df.prob

In [227]:
groups = df_triples.groupby(by=['head','tail'])
df_filter = groups.apply(lambda g: g[g['prob'] == g['prob'].max()])
df_filter[df_filter.prob > 0.9999].to_csv('./sorted_BERT_direct.csv', index = False)